<a href="https://colab.research.google.com/github/elpiprin/NLP-Stuff/blob/main/RNN_Custom3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

***Imports***

In [1]:
#Source: https://www.tensorflow.org/text/tutorials/text_generation
import tensorflow as tf
import numpy as np
import os
import time

***Reading the data***

In [2]:
#Read, then decode for py2 compat.
text = open('/content/Articles.txt', 'rb').read().decode(encoding='utf-8')
#length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 480143 characters


In [3]:
#Take a look at the first 250 characters in text
print(text[:250])

Melissa Caddick: Death of Australian fraudster remains a mystery

When conwoman Melissa Caddick vanished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed foot found washed up on a beach months later - it 


In [4]:
#The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

105 unique characters


### **Processing the text**

***Vectorising the text***

In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

***Prediction task*** 

***Creating training examples and targets*** 

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(480143,), dtype=int64, numpy=array([43, 63, 70, ..., 76, 77, 16])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

M
e
l
i
s
s
a
 
C
a


In [15]:
seq_length = 100

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'M' b'e' b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c'
 b'k' b':' b' ' b'D' b'e' b'a' b't' b'h' b' ' b'o' b'f' b' ' b'A' b'u'
 b's' b't' b'r' b'a' b'l' b'i' b'a' b'n' b' ' b'f' b'r' b'a' b'u' b'd'
 b's' b't' b'e' b'r' b' ' b'r' b'e' b'm' b'a' b'i' b'n' b's' b' ' b'a'
 b' ' b'm' b'y' b's' b't' b'e' b'r' b'y' b'\r' b'\n' b'\r' b'\n' b'W' b'h'
 b'e' b'n' b' ' b'c' b'o' b'n' b'w' b'o' b'm' b'a' b'n' b' ' b'M' b'e'
 b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c' b'k' b' '
 b'v' b'a' b'n'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'
b'ished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed fo'
b'ot found washed up on a beach months later - it set off a frenzy in Australia.\r\n\r\nThe case blindsided'
b' investors, baffled police, and captured the imagination of a nation.\r\n\r\nThe fraudster has inspired a'
b' hit podcast, a TV dramatisation, and countless outlandish theories - including that she had been swa'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick va'
Target: b'elissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'


***Creating training batches***

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Building the model**

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Trying the model**

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27136     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  108650    
                                                                 
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([103,  81,  32,  52,  40,  86, 105,  49,   2,  28,  59,  50,  89,
        59,   7,  73, 103,   0,  49,  82,  11,  52,  18,  42,   4,  20,
        25,  79,  33,  23,  28,  84,  36,  74,  26,  93,  99,  84,  66,
        28,  58,  41,  91,  75,  40,  51,  59,  11,  97,  84,  23,  70,
         6,  95,  37,  44,  83,  50,  77,  30, 102,  84,  75,  55,  88,
        12,  14,   9,  76,  11,  44,  49,  82,  88,  93,  87,  61,  15,
        98,   9,  73,  24,  92,  26,  76,   8,  24,  91,  63,  25, 102,
        99,  30,  56,  42,  71,  66,  98,   2,  79])

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'Under reforms made to Qatar\xe2\x80\x99s labour laws in 2020, employers and workers can end contracts early, bu'

Next Char Predictions:
 b'\xe2\x80\xa6wBVJ\xc3\xa1\xe2\x82\xacS\r:aT\xc3\xa9a$o\xe2\x80\xa6[UNK]Sx(V0L!27uC5:zFp8\xc3\xb6\xe2\x80\x98zh:]K\xc3\xb1qJUa(\xe2\x80\x93z5l#\xc3\xbcGNyTs?\xe2\x80\x9dzqY\xc3\xa8),&r(NSx\xc3\xa8\xc3\xb6\xc3\xa3c-\xe2\x80\x94&o6\xc3\xb38r%6\xc3\xb1e7\xe2\x80\x9d\xe2\x80\x98?ZLmh\xe2\x80\x94\ru'


### **Training the model**

***Attaching an optimiser & a loss function***

In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6626196, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

105.91317

In [34]:
model.compile(optimizer='adam', loss=loss)

***Configuring checkpoints***

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

***Executing the training***

In [36]:
EPOCHS = 60

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
74/74 [==============================] - 410s 5s/step - loss: 3.1432
Epoch 2/60
74/74 [==============================] - 407s 5s/step - loss: 2.3965
Epoch 3/60
74/74 [==============================] - 407s 5s/step - loss: 2.2080
Epoch 4/60
74/74 [==============================] - 412s 6s/step - loss: 2.0481
Epoch 5/60
74/74 [==============================] - 406s 5s/step - loss: 1.9003
Epoch 6/60
74/74 [==============================] - 411s 6s/step - loss: 1.7557
Epoch 7/60
74/74 [==============================] - 406s 5s/step - loss: 1.6282
Epoch 8/60
74/74 [==============================] - 410s 6s/step - loss: 1.5228
Epoch 9/60
74/74 [==============================] - 404s 5s/step - loss: 1.4362
Epoch 10/60
74/74 [==============================] - 401s 5s/step - loss: 1.3637
Epoch 11/60
74/74 [==============================] - 408s 6s/step - loss: 1.3008
Epoch 12/60
74/74 [==============================] - 403s 5s/step - loss: 1.2438
Epoch 13/60
74/74 [==================

### **Generating Text**

In [69]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.7):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [70]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [72]:
#Text 1
start = time.time()
states = None
next_char = tf.constant(['Activists'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Activists have long said they want to see abortion providers, back at Sanyiag-neth language as a Germany.

"The process of creating the other is by propagating the idea that if we will come home alive or not,” he added.

Analysts at Bank of America and Goldman Sachs, for their part, see summer.

On the Republican figures serving pulled international law, and that the UK was reduced to a couple of cells in the country.

Ms Hawkins read the conclusion the work compliance with political dictates,” Thomas added to access medication through an online provider," said Elizabeth Nash, a political analyst. "The system is collapsing in Pakistan's constitution. Internet access is a human behind the allegations say about the condition of these airplanes, if the mandatory reporting looks as they are for the developing countries where they are being given does not allow the health system to pay the doctors what they need,” says Djalo. “If someone has a cardiovascular illness, they must apply forces 

In [73]:
#Text 2
start = time.time()
states = None
next_char = tf.constant(['President'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

President on Thursday calling on him to invoke the 14th Amendment.

Senate Majority Whip Dick Durbec and The seals, contested Nazi Germany in an anti-vaccine speech at the Lincoln Memorial powers or people who are severely immunocompromised may he in late the education nets of the party, is now chanceller and activists said the banner of the Congress party, were offered in the case with Lieberman.

In the last decade, CNN has documented multiple cases have been reported during 737 Max flights.

Boeing insists the protesters with the impeachment a “politically motivated to produce a lot of love from my friends. But now, it’s weird, but I do feel more complete. [I feel] loved in the area.

Deminers have arrived in the town this year from Wajer and American women had a constitutional right to an abortion - and they were not given important roless, which has never heard a word of hatred from his mouth. He could be angry with someone but keeps civilians have become so effective.

“We place 

In [76]:
#Text 3
start = time.time()
states = None
next_char = tf.constant(['Conflict'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Conflicting court in a particular area change in tour. She argues that if both sides of the aisle – to get the bill over the finish line.

McCarthy said the broader context of last week’s all-day course of treatment, to see if that can be decided in the outside Afghanistan.”

The US has not particularly dragons were taken to a museum. Although all the children reached for the immediate release of the three men, Equidem’s director, Mustafa Qadri, who were often around in the way. Your only aim is 54 years old, Arnold failed to properly address their concerns.

The findings has vounced the rape of where Africa goes, the transition shouldn’t necessarily be from used cars to brand new combustion.

The 331st were accused of massacring hundreds of other serious crisis," says Michael Kugelman, director for the Imminist which the allegations against Hari Padman first surfaced in December in a now-deleted Facebook comments. “Racism exists here but I don’t agree Spain. The scheme will also requi

In [81]:
#Text 4
start = time.time()
states = None
next_char = tf.constant(['Asylum'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Asylum-seekers sleep in a hospital lobby

Fleeing what she was cornered by a campul and the travel go online tank targets.

“The viruses may be competing with each other,” she said. “But we should be very careful about both time to protect them from hail damage. Now they are experimenting with nets that can protect against the clock because he’s telling me that he’s reaching out to all these other communities, as we saw recently in Mayhill campaign to reach 2.2 million people.

Unicef has the season and intelligent the perception affect the country's prospects of ocean warming ahead of El Niño alarms scientists

According to the most recent estimates, more than 65 million views on YouTube alone.

Viewerman family members who he said were on hand Wednesday, including children, grandchildren, siblings and communities say that provided the pills of the three repertory artists had been "dispected with a central player in the AI revolution?

If the best initiatives are “when media outlets o

In [83]:
#Text 5
start = time.time()
states = None
next_char = tf.constant(['Corruption'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Corruption crackdown is Xi’s signature campaign. It has swept across the ruling Communist Party, the group’s second strike in Kabul that killed 10 civilians, including many of her family and friends, to help fund a later decades of his edie.

“Seeking the allegations against the brothers have sent shockwaves through the classical music world, where they entire wing ‘one an acto.

Mr Ali says he hopes that some day, justice will also be delivered in time for Ukraine's expected counter-offensive. President Volodymyr Zelensky has called "cow smuggling or transportation," althorities as a place to sell their older gas-powered models, according to the Senate. The deal must get passed before June 5 – the crucial in students who come to school knowing another language.

The pilots say they are offering concrete suggestions and want to talk to power more than 1.5C since preindustrial times. This is bas crowledged they didn’t know much about Kennedy’s environmental agenda, and the acute crisis 

In [84]:
#Text 6
start = time.time()
states = None
next_char = tf.constant(['Investigation'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Investigation about the impact of these conspiracies on climate politics. “If we want to implying to the weeked officials about putting in new regulations that would crack down on shipping very old or junk.

"I used to be healthy, walk with both of my feel a former senior safety engineer at the FAA, which has been deeply critical of the property our air defense system almost every night.”

“We are spending our missiles and ammunition and extremism.

The place where it they posted images of blue tarpaulin sheets that serves appairnt, the company and authorities cheated us,” said another fired security guard from Pakistan.

Despite the hours of air time, ferociously delivered ultimatums and street stand-offsite the conflict, drawing the US and Europe directly into the war.

And even training Ukraine's military populations that the United States’ rise to global domination sometimes seemed on the brink of collapsient in prison under Hari Padman and the federal government was in conflict wi

In [101]:
#Text 7
start = time.time()
states = None
next_char = tf.constant(['Official'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Officials and performance reviews. There is no meritocracy.”

A document costing the Taliban’s takeover, killing at least 670 people and injuring 1,200 — a significant uptime at his Senate trial.

Lieberman’s story starts in late 1979 in the town of California, New York and Connecticut have targeted the bill would include an administrative pay-as-you-go provision to require that Biden said in a statement. “Nevertheless, I had multiple interactions with the speaker through my time as a huge shock.

"I'm not surprised dependents.

The 737 Max remains under intense scrutiny. The plane was grounded for 30 others. What was once an historic work space for them. CNN has reached out to the agency for comment.

Turned away, is a Biden debate is that sometimes these abuses are from our very counterplated up their strengthing of codon.

Several scinns in communist East Germany’s controversial figural values are the air and oceans.

Another important factor that is worrying scientists remain unpai

In [88]:
#Text 8
start = time.time()
states = None
next_char = tf.constant(['Pandemic'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Pandemic Stagism in February 2023 by the Russian network NTV shows an "armoured group" of the 331st in action in Luhan and Asia brother post-accused of theft and many jailed.

Three postmastry said there had been investment, and the community had a very good youth centre, leading and was the only doctor left for the army - lite a daming threat of economic catastrophe.

In March 2022, the Utica Schools Board bought Evolv's weapons scanning system for Malyana violations.

The BBC couldn't contact Mr Metalfase of Seputails and congested may count in the near issue is that she’s wanted to own their strength to be able to withstand the "brutal level" that awaits them. Mr Sengupta calls Metwore has now been handed available options, like power in Germany say they have reached their limits, and local authorities are finding it difficult to move at short notice, he's not put off by the month-to-month living surgercian, and the JA establi has been directed at any black or non-white bringing in 

In [93]:
#Text 9
start = time.time()
states = None
next_char = tf.constant(['Scientist'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Scientists point to collect health information. “There’s no benefit to NHS trusts many students from Kenya’s hospitals have reported an increase in patient numbers in recent months, but at least the lower in the past 10 years were Muslims.

And the articles say voters in Germany say they have reached the conclusion that he was the man to find A healthy setting request for new IRS agents, and claw back tens of billions in unspent funds meant in a figure largern officers as scheduled to baly to the point where the principals can meet and detained as a "fresh start for Scotland" after a tumultuous threar young people - but without undermining Scotland's controversial deporting in Covid-19 and the cost of living crisis.

He believes the more restrictions will follow.

In recent days, abortion - including medical to be her own perception problem.

Aide thousands of people from long Covid and ME/CFS are diseases of global significance. It’s valuation into the severity that brought in the com

In [96]:
#Text 10
start = time.time()
states = None
next_char = tf.constant(['Global'])
result = [next_char]

for n in range(11000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Global Indian music gurus accused of sexual assault

Her district — a scientists leading the next 18 may could an obviously tempting target.”

Of course, there are all the usual issues that a strong El Niño weather event - a weather system in the hospital because she was malnourished. Hussein fetches a 20-litre barrel of water from the 331st remembers the Great Patrick, has been charged and trust through instead to tell yet. Some would else and the trieb in decades, the relationship between ITV's security spracks, and now the agency describes itself as a window through her parties decide what their red lines are." She argues that if both sides are experiencing dangerous.

"These drugs have caused injury, infertility, death," Ms Hawkins said Catholican helping to “grow your business”. In one guide of the decision ob border in the classroom rather than anything I’d had before the police and captured the persent of a broader shift away from corporate speak".

So what changed to make a con

### **Exporting the generator**

In [97]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [98]:
states = None
next_char = tf.constant(['Trial'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Trial called the “mandatties such as Japan, South Korea, European countries and, increasingly, the US. Ma
